# Intermediate Scraping Homework: Wikipedia Table

In this assignment, we'll be extracting data from Wikipedia's table of the tallest buildings in Brooklyn: https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn

### 0) Setup

Import `requests`, `BeautifulSoup`, and `pandas`. Although this homework uses `BeautifulSoup`, you can choose to use `lxml` instead, if you prefer.

In [154]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 1) Grab the HTML for the webpage linked above

Use `requests` to get the HTML, assigning it to a variable

In [155]:
http_response = requests.get("https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn")

# store the HTML of the page in a variable
html = http_response.text

### 2) Convert the HTML into a `BeautifulSoup` object

In [156]:
# convert the HTML to DOM representation using BeautifulSoup
soup = BeautifulSoup(html)

### 3) Use `.select(...)` (and `[0]`) to select the main table

That's the one directly under the "Tallest buildings" heading.

Print out the first 100 characters of text from the table to make sure you have the right one.

In [157]:
# find the first table in the HTML
table = soup.select("table")[0]
print(table.text[:100])

# or find the first table with the class "wikitable"
# table = soup.select(".wikitable")[0]
# print(table.text[:100])





Rank

Name[a]

Image

Heightft (m)

Floors

Year completed

Notes


1

The Brooklyn Tower



1,066


### 4) Use `.select(...)` (and `[0]`) again to select the table's first row

... which is its header. (Reminder that the `<thead>` tag is optional. Wikipedia tables don't use it.)

In [158]:
first_row = table.select("tr")[0]
print(first_row.text)


Rank

Name[a]

Image

Heightft (m)

Floors

Year completed

Notes



### 5) Extract the column names from that header

Use `.strip()` to remove any leading or trailing whitespace from the names.

First, try doing this with a standard `for` loop:

In [159]:
column_names = []

for th in first_row.select("th"):
    column_names.append(th.text.strip())

print(column_names)

['Rank', 'Name[a]', 'Image', 'Heightft (m)', 'Floors', 'Year completed', 'Notes']


Try to do the same, but with a list comprehension:

In [160]:
column_names = [th.text.strip() for th in first_row.select("th")]
print(column_names)

['Rank', 'Name[a]', 'Image', 'Heightft (m)', 'Floors', 'Year completed', 'Notes']


### 6) Select all non-header row *elements* from the table

Since the header was the first row, you'll want to skip that one. How many rows are there? (Check with your eyes that this number matches what you deduce from the rankings column in the browser-rendered table.)

In [161]:
### 6) Select all non-header row *elements* from the table
row_not_header = table.select("tr")[1:]
print(len(row_not_header))

80


### 7) For each row, extract the text of each cell into a Python list

First, try this as two nested `for` loops:

In [162]:
cell_textes = []
for row in row_not_header:
    cell_texts = []
    for cell in row.select("td"):
        cell_texts.append(cell.text.strip())
    cell_textes.append(cell_texts)
print(cell_textes)

[['1', 'The Brooklyn Tower', '', '1,066 (325)', '93', '2022', 'Topped out in October 2021.[2][23][24][25]'], ['2', 'Brooklyn Point', '', '720 (219)', '68', '2019', "The final phase of Extell's City Point development; topped out in April 2019, it is now the second tallest building in Brooklyn.[26] Also known as 138 Willoughby Street,[27][28] 1 City Point,[29] and City Point Tower III.[29][30][31]"], ['3', '11 Hoyt', '', '626 (191)', '51', '2020', "Topped out in June 2019.[32] A redevelopment of Macy's former footprint in Downtown Brooklyn, with a design seemingly inspired by 8 Spruce Street.[33][34]"], ['4', 'The Hub', '', '611 (186)', '52', '2017', 'Also known as 333 Schermerhorn Street. Topped out on December 16, 2015.[35][36][37][38][39]'], ['5', '98 Dekalb Avenue', 'Upload image', '610 (186)', '49', '2025', 'Topped out in June 2024.[40][41]'], ['6', 'AVA DoBro', '', '596 (182)', '58', '2015', 'Also known as 100 Willoughby Street, Avalon Willoughby Square, and 214 Duffield Street.[42

Try the same, but with two list comprehensions (one nested in the other):

In [163]:
cell_textes = [[cell.text.strip() for cell in row.select("td")] for row in row_not_header]
print(cell_textes)

[['1', 'The Brooklyn Tower', '', '1,066 (325)', '93', '2022', 'Topped out in October 2021.[2][23][24][25]'], ['2', 'Brooklyn Point', '', '720 (219)', '68', '2019', "The final phase of Extell's City Point development; topped out in April 2019, it is now the second tallest building in Brooklyn.[26] Also known as 138 Willoughby Street,[27][28] 1 City Point,[29] and City Point Tower III.[29][30][31]"], ['3', '11 Hoyt', '', '626 (191)', '51', '2020', "Topped out in June 2019.[32] A redevelopment of Macy's former footprint in Downtown Brooklyn, with a design seemingly inspired by 8 Spruce Street.[33][34]"], ['4', 'The Hub', '', '611 (186)', '52', '2017', 'Also known as 333 Schermerhorn Street. Topped out on December 16, 2015.[35][36][37][38][39]'], ['5', '98 Dekalb Avenue', 'Upload image', '610 (186)', '49', '2025', 'Topped out in June 2024.[40][41]'], ['6', 'AVA DoBro', '', '596 (182)', '58', '2015', 'Also known as 100 Willoughby Street, Avalon Willoughby Square, and 214 Duffield Street.[42

### 8) Turn the data you've extracted into a `pandas` `DataFrame`

In [164]:
df = pd.DataFrame(cell_textes, columns=column_names)
df

,Rank,Name[a],Image,Heightft (m),Floors,Year completed,Notes
0,1,The Brooklyn Tower,,"1,066 (325)",93,2022,Topped out in October 2021.[2][23][24][25]
1,2,Brooklyn Point,,720 (219),68,2019,The final phase of Extell's City Point develop...
2,3,11 Hoyt,,626 (191),51,2020,Topped out in June 2019.[32] A redevelopment o...
3,4,The Hub,,611 (186),52,2017,Also known as 333 Schermerhorn Street. Topped ...
4,5,98 Dekalb Avenue,Upload image,610 (186),49,2025,Topped out in June 2024.[40][41]
...,...,...,...,...,...,...,...
75,76,595 Dean Street,Upload image,298 (91),27,2022,Topped out in February 2022.[178][179]
76,77 =,Beacon Tower,,297 (91),23,2007,[180][181]
77,77 =,One Northside Piers,,297 (91),29,2008,[182][183]
78,77 =,101 Clark Street,Upload image,295 (90),30,1973,[184][185]


### 9) Which years are represented by at least 5 buildings?

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Rank            80 non-null     object
 1   Name[a]         80 non-null     object
 2   Image           80 non-null     object
 3   Heightft (m)    80 non-null     object
 4   Floors          80 non-null     object
 5   Year completed  80 non-null     object
 6   Notes           80 non-null     object
dtypes: object(7)
memory usage: 4.5+ KB


In [166]:
# We need to count the number of buildings for each year
year_counts = df["Year completed"].value_counts()


# Finally, we can filter the years that have at least 5 buildings
years_with_5_buildings = year_counts[year_counts >= 5]

years_with_5_buildings

Year completed
2022    11
2020     6
2016     6
2024     5
Name: count, dtype: int64

### 10) How many total floors do all the buildings have, combined?

In [167]:
# column floors to integer
df['Floors'] = df['Floors'].str.extract('(\d+)').astype(int)

# sum of floors
total_floors = df['Floors'].sum()
total_floors

2905

### 11) How many of the buildings have their own Wikipedia page?

For this, you'll need to query the row elements again; the information won't have been extracted into your `DataFrame`. 

(Hint: Whether a building has its own Wikipedia page isn't an explicit piece of data, but something you can infer from the presence of a particular sub-element.)

In [175]:
links = []
for row in row_not_header:
    cell = row.select("td")[1]  # select the second td element
    link = cell.select_one("a")  # if a link exists in cell
    if link:  # If such a link exists
        links.append(1)  # Append 1 to links, indicating a match
print(len(links))

18


In [179]:
links = 0
for row in row_not_header:
    cell = row.select("td")[1]  # select the second td element
    link = cell.select("a")  # if a link exists in cell
    if link:  # If such a link exists increment links by 1
        links += 1
print(links)

18


### 12) How many have an image?

You could do this by testing for the presence of another element:

In [180]:
# Find the index of the "Image" column
# image_index = column_names.index("Image")

# Initialize a counter
image_count = 0

# Loop over the rows
for row in row_not_header:
    # Select the cell in the "Image" column (the third column)
    cell = row.select("td")[2]
    # Check if the cell contains an image tag
    if cell.select("img"):
        image_count += 1

print(image_count)


61


Or through information that's already in your `DataFrame`:

In [170]:
# Count the number of rows where the "Image" column is not "Upload image"
image_count = (df["Image"] != "Upload image").sum()

print(image_count)


61


### Bonus challenge

If we tried to run the same code on https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_New_York_City instead, the results wouldn't be quite right. Try it. Then, examining the HTML of that page, try to figure out why.

If you want an extra-extra challenge, try writing code that would parse that table correctly.

In [182]:
# Load the page
url = "https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_New_York_City"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table - assuming it's the first table of class 'wikitable'
table = soup.find('table', {'class': 'wikitable'})

# Initialize a list to store row data
rows_list = []

# Process table headers
headers = [header.text.strip() for header in table.find_all('th')]

# Iterate over table rows
for row in table.find_all('tr'):
    cells = row.find_all(['td', 'th'])
    if cells:
        row_data = []
        for cell in cells:
            # Handle cell text and cleanup
            cell_text = cell.get_text(strip=True)
            # Optionally, handle 'rowspan'/'colspan'
            row_data.append(cell_text)
        rows_list.append(row_data)

# Convert list of rows into a DataFrame
df = pd.DataFrame(rows_list, columns=headers)
df


,Rank,Name,Image,Heightft (m),Floors[H],Year,Address,Coordinates,Notes
0,Rank,Name,Image,Heightft (m),Floors[H],Year,Address,Coordinates,Notes
1,1,One World Trade Center,,"1,776 (541)",94[A],2014,285Fulton Street,40°42′47″N74°00′49″W﻿ / ﻿40.713°N 74.0135°W﻿ /...,Also known as the Freedom Tower. Tallest build...
2,2,Central Park Tower,,"1,550 (472)",99,2021,225 West57th Street,40°45′57″N73°58′51″W﻿ / ﻿40.7659°N 73.98089°W﻿...,"Also known asNordstromTower. At 1,550 feet, it..."
3,3,111 West 57th Street,,"1,428 (435)",85,2022,111 West57th Street,40°45′52″N73°58′40″W﻿ / ﻿40.76455°N 73.97765°W...,Also known asSteinwayTower. It is the world'sm...
4,4,One Vanderbilt,,"1,401 (427)",73,2020,1Vanderbilt Avenue,40°45′11″N73°58′43″W﻿ / ﻿40.7530°N 73.9785°W﻿ ...,Second-tallest office building in NYC.[38]Tall...
...,...,...,...,...,...,...,...,...,...
105,105,1 Wall Street,,654 (199),50,1932,1Wall Street,40°42′26″N74°00′42″W﻿ / ﻿40.707222°N 74.011667...,It was formerly calledBank of New YorkBuilding...
106,106 =,599 Lexington Avenue,,653 (199),51,1986,599Lexington Avenue,40°45′28″N73°58′15″W﻿ / ﻿40.7578°N 73.9707°W﻿ ...,[247][248]
107,106 =,Silver TowersI,,653 (199),58,2009,620 West42nd Street,40°45′39″N73°59′57″W﻿ / ﻿40.760722°N 73.999194...,Also known as River Place.[249][250]
108,106 =,Silver TowersII,653 (199),58,2009,620 West42nd Street,40°45′39″N73°59′57″W﻿ / ﻿40.760722°N 73.999194...,Also known as River Place.[251][252],None


---

---

---